In [52]:
import nfl_data_py as nfl
import sqlite3
import pandas as pd
import numpy as np

In [53]:
# Connect to your SQLite DB (will create if it doesn't exist)
conn = sqlite3.connect(r'C:\Users\tx400\SQL_Downloads\nfl_data_db.sqlite')

#### Read in pbp and ftn tables

In [54]:
# Import pbp >= yr 2022
df = pd.read_sql('SELECT * FROM pbp_stats_df WHERE season >=2022', conn)

In [55]:
# Import ftn dfs
df2 = pd.read_sql('SELECT * FROM ftn_df', conn)

#### Merge pbp and ftn tables

In [56]:
# Left join pbp_df and ftn_df
pbp_v2 = df.merge(
    df2,
    how='left',
    left_on=['game_id', 'play_id'],
    right_on=['nflverse_game_id', 'nflverse_play_id']
)

In [57]:
pbp_v2.tail()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week_x,posteam,posteam_type,defteam,...,read_thrown,is_catchable_ball,is_contested_ball,is_created_reception,is_drop,is_qb_sneak,n_blitzers,n_pass_rushers,is_qb_fault_sack,date_pulled
99094,4791.0,2023_22_SF_KC,None,KC,SF,POST,22,KC,home,SF,...,0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2024-09-06 23:50:14.696374
99095,4813.0,2023_22_SF_KC,None,KC,SF,POST,22,KC,home,SF,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-09-06 23:50:14.696374
99096,4835.0,2023_22_SF_KC,None,KC,SF,POST,22,KC,home,SF,...,DES,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2024-09-06 23:50:14.696374
99097,4860.0,2023_22_SF_KC,None,KC,SF,POST,22,KC,home,SF,...,1,1.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2024-09-06 23:50:14.696374
99098,4881.0,2023_22_SF_KC,None,KC,SF,POST,22,KC,home,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Drop all unwanted columns

In [58]:
# Copy and paste columns you want to keep from excel
raw = """
ftn_game_id
starting_hash
qb_location
n_defense_box
is_no_huddle
is_motion
is_play_action
is_screen_pass
is_rpo
is_trick_play
game_half
play_deleted
is_qb_out_of_pocket
is_interception_worthy
is_throw_away
read_thrown
is_catchable_ball
is_contested_ball
is_created_reception
is_drop
is_qb_sneak
n_blitzers
n_pass_rushers
is_qb_fault_sack
timeout
home_opening_kickoff
quarter_end
play_id
game_id
home_team
away_team
season_type
posteam
defteam
yardline_100
game_date
drive
qtr
down
goal_to_go
ydstogo
ydsnet
play_type
yards_gained
shotgun
no_huddle
qb_dropback
qb_scramble
pass_length
pass_location
air_yards
yards_after_catch
run_location
run_gap
field_goal_result
kick_distance
extra_point_result
two_point_conv_result
td_team
td_player_name
td_player_id
score_differential
ep
epa
wp
wpa
first_down_rush
first_down_pass
third_down_converted
fourth_down_converted
incomplete_pass
interception
punt_inside_twenty
punt_in_endzone
punt_out_of_bounds
kickoff_inside_twenty
kickoff_in_endzone
kickoff_out_of_bounds
fumble_forced
fumble_not_forced
solo_tackle
safety
tackled_for_loss
fumble_lost
qb_hit
rush_attempt
pass_attempt
sack
touchdown
pass_touchdown
rush_touchdown
return_touchdown
extra_point_attempt
two_point_attempt
field_goal_attempt
kickoff_attempt
punt_attempt
fumble
complete_pass
assist_tackle
passer_player_id
passer_player_name
passing_yards
receiver_player_id
receiver_player_name
receiving_yards
rusher_player_id
rusher_player_name
rushing_yards
interception_player_id
interception_player_name
punt_returner_player_id
punt_returner_player_name
kickoff_returner_player_name
kickoff_returner_player_id
punter_player_id
punter_player_name
kicker_player_name
kicker_player_id
blocked_player_id
blocked_player_name
tackle_for_loss_1_player_id
tackle_for_loss_1_player_name
tackle_for_loss_2_player_id
tackle_for_loss_2_player_name
qb_hit_1_player_id
qb_hit_1_player_name
qb_hit_2_player_id
qb_hit_2_player_name
forced_fumble_player_1_player_id
forced_fumble_player_1_player_name
forced_fumble_player_2_player_id
forced_fumble_player_2_player_name
solo_tackle_1_player_id
solo_tackle_2_player_id
solo_tackle_1_player_name
solo_tackle_2_player_name
assist_tackle_1_player_id
assist_tackle_1_player_name
assist_tackle_2_player_id
assist_tackle_2_player_name
assist_tackle_3_player_id
assist_tackle_3_player_name
assist_tackle_4_player_id
assist_tackle_4_player_name
tackle_with_assist_1_player_id
tackle_with_assist_1_player_name
tackle_with_assist_2_player_id
tackle_with_assist_2_player_name
pass_defense_1_player_id
pass_defense_1_player_name
pass_defense_2_player_id
pass_defense_2_player_name
fumbled_1_player_id
fumbled_1_player_name
fumbled_1_team
fumbled_2_player_id
fumbled_2_player_name
fumbled_2_team
fumble_recovery_1_team
fumble_recovery_1_yards
fumble_recovery_1_player_id
fumble_recovery_1_player_name
fumble_recovery_2_team
fumble_recovery_2_yards
fumble_recovery_2_player_id
fumble_recovery_2_player_name
sack_player_id
sack_player_name
half_sack_1_player_id
half_sack_1_player_name
half_sack_2_player_id
half_sack_2_player_name
penalty_team
penalty_player_id
penalty_player_name
penalty_yards
replay_or_challenge
replay_or_challenge_result
penalty_type
safety_player_name
safety_player_id
cpoe
series
series_success
series_result
start_time
stadium
weather
play_clock
special_teams_play
st_play_type
drive_play_count
drive_time_of_possession
drive_first_downs
drive_inside20
drive_ended_with_score
drive_yards_penalized
away_score
home_score
temp
wind
home_coach
away_coach
stadium_id
game_stadium
passer
rusher
receiver
pass
rush
first_down
special
play
passer_id
rusher_id
receiver_id
offense_formation
offense_personnel
defenders_in_box
defense_personnel
number_of_pass_rushers
time_to_throw
was_pressure
route
defense_man_zone_type
defense_coverage_type

"""

In [59]:
# Convert the above excel column into a python list through list comprehension
keep_cols = [line.strip() for line in raw.strip().splitlines() if line.strip()]

#### Reorder columns such that groupings have more affinity

In [60]:
# Drop all unwanted columns from pbp_v2
pbp_v2 = pbp_v2[keep_cols]

# Read in excel file to reorder column names
order_df = pd.read_excel("pbp_v2_column_reorder.xlsx")

# Reorder column names
col_order_list = order_df['column_name'].tolist()
pbp_v2 = pbp_v2[col_order_list]

#### Join players position group and position to pbp from players df

In [61]:
# Read in players df
df3 = pd.read_sql('SELECT * FROM players_df', conn)

In [62]:
# Create a list of players_ids you would like to merge with
id_cols = """
td_player_id
passer_player_id
passer_id
receiver_player_id
receiver_id
rusher_player_id
rusher_id
interception_player_id
punt_returner_player_id
kickoff_returner_player_id
punter_player_id
kicker_player_id
blocked_player_id
tackle_for_loss_1_player_id
tackle_for_loss_2_player_id
qb_hit_1_player_id
qb_hit_2_player_id
forced_fumble_player_1_player_id
forced_fumble_player_2_player_id
solo_tackle_1_player_id
solo_tackle_2_player_id
assist_tackle_1_player_id
assist_tackle_2_player_id
assist_tackle_3_player_id
assist_tackle_4_player_id
tackle_with_assist_1_player_id
tackle_with_assist_2_player_id
pass_defense_1_player_id
pass_defense_2_player_id
fumbled_1_player_id
fumbled_2_player_id
fumble_recovery_1_player_id
fumble_recovery_2_player_id
sack_player_id
half_sack_1_player_id
half_sack_2_player_id
penalty_player_id
safety_player_id
""".strip().split("\n")

# merge each player id within a play to there player group and position via players_df
pbp_v3 = pbp_v2.copy()

for col in id_cols:
    pbp_v3 = pbp_v3.merge(
        df3[['gsis_id', 'position_group', 'position']],
        how='left',
        left_on=col,
        right_on='gsis_id'
    ).drop(columns=['gsis_id'])  # drop merge key

    # Rename the merged columns so they are unique
    pbp_v3 = pbp_v3.rename(columns={
        'position_group': f"{col.removesuffix('_id')}_position_group",
        'position': f"{col.removesuffix('_id')}_position"
    })


In [63]:
#pbp_v3.tail()


In [64]:
# Read in excel file to reorder column names
order_df2 = pd.read_excel("pbp_v3_column_reorder.xlsx")

# Reorder column names
col_order_list = order_df2['column_name'].tolist()
pbp_v3 = pbp_v3[col_order_list]

In [65]:
# Upload the table to nfl_data_db
pbp_v3.to_sql('pbp_v3', conn, if_exists='replace', index=False)

99099

In [66]:
pbp_v3.shape

(99099, 303)

In [67]:
(pbp_v3['passer_id'] == pbp_v3['passer_player_id']).all()

False

In [68]:
conn.close

<function Connection.close()>